In [1]:
# Run on GPU server
import sys
sys.path.append("../StockPriceForecast/")

In [2]:
import torch
import os
print("Cuda is available : ", torch.cuda.is_available())
print("Cuda num : ", torch.cuda.device_count())  # gpu数量
print("Current cuda index : ", torch.cuda.current_device())
print("Current cuda name : ", torch.cuda.get_device_name(0))

Cuda is available :  True
Cuda num :  1
Current cuda index :  0
Current cuda name :  NVIDIA GeForce RTX 3090


In [3]:
from libs import *
dataPath = "../StockPriceForecast/dataSet/data/marketData/data.nc"
data_len = DataLength(dataPath)
print("All days : ", data_len)
input_days = 360
predict_days = 30
test_predict_len = 360  # 预测360天
train_dataSet = DataSet(dataPath=dataPath, isel=[0, data_len - input_days - test_predict_len])
print("Train days : ", train_dataSet.data.shape)
validation_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + 2 * test_predict_len), data_len - test_predict_len])
print("Validation days ; ", validation_dataSet.data.shape)
test_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + test_predict_len), data_len])
print("Test days : ", test_dataSet.data.shape)

train_dataLoader = DataLoader(train_dataSet)
validation_dataLoader = DataLoader(validation_dataSet)
test_dataLoader = DataLoader(test_dataSet)

All days :  2994
Train days :  (2274, 1440, 10)
Validation days ;  (720, 1440, 10)
Test days :  (720, 1440, 10)


In [4]:
# train_dataSet = train_dataSet.isel(0, 395)
# train_dataLoader = DataLoader(train_dataSet)

from libs.modules import *
from libs import *
import torch
import torch.nn as nn
model = ResNetSeq2SeqAttention(use_checkpoint=True,
                                resNet_layer_nums=[3,4,6,3],
                                output_size=1440,
                                seq2seq_hidden_size=2048,
                                seq2seq_num_layer=2)
# device_ids = [0, 1]
# model = torch.nn.DataParallel(model, device_ids=device_ids)
model.train()
device = torch.device("cuda:0")
model, attention_weight_seq, Loss =train_ResNetSeq2SeqAttention(model=model,
                    dataLoader=train_dataLoader, learning_rate=0.00001,
                    alpha=0.3, gamma=0.5, device=device, num_epoch=1, init_weights=True)
torch.save(model, "../StockPriceForecast/Models/spf_5.pth")

#  由于ResNet使用了半精度，显存占用降至之前的一半(12GB)，可以适当降低 checkpoint_sequential 的 chunks 以提高训练速度
#  由于在训练过程中显存占用逐渐加大，最终爆显存，所以需要加入模型保存和优化器保存，能够继续训练

  0%|          | 1/1885 [00:01<41:14,  1.31s/it]

loss :  tensor(0.1382, grad_fn=<ToCopyBackward0>)


  0%|          | 2/1885 [00:02<41:14,  1.31s/it]

loss :  tensor(0.1132, grad_fn=<ToCopyBackward0>)


  0%|          | 3/1885 [00:03<40:06,  1.28s/it]

...

loss :  tensor(0.0013, grad_fn=<ToCopyBackward0>)


100%|█████████▉| 1884/1885 [39:48<00:01,  1.27s/it]

loss :  tensor(0.0013, grad_fn=<ToCopyBackward0>)


100%|██████████| 1885/1885 [39:50<00:00,  1.27s/it]

loss :  tensor(0.0013, grad_fn=<ToCopyBackward0>)
latest loss 0.0012923454632982612, cost 2390.102600812912 sec on cuda:0


In [5]:
a = torch.cuda.memory_reserved(0)
print(a)

7289700352


In [13]:
model.eval()
for data, tar in validation_dataLoader:
    data.cuda()
    data = data.reshape(360, 1, 120, 120)
    out = model(data)
    break
print(out)

TypeError: exceptions must derive from BaseException